In [12]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import os # getting filepath
import copy
import matplotlib.pyplot as plt
import math

import more_itertools as mit
import sqlite3

In [13]:
def find_ranges(iterable):
    """Yield range of consecutive numbers."""
    for group in mit.consecutive_groups(iterable):
        group = list(group)
        if len(group) == 1:
            yield group[0]
        else:
            yield group[0], group[-1]

In [14]:
conn = sqlite3.connect('fursealdata.db')
c = conn.cursor()

# COMMENT IF YOU DON'T WANT THE TABLE DROPPED EACH TIME
# (The .db needs to be created anew anytime you give it a different directory of files)
# Otherwise, change the line containing "CREATE TABLE fursealdata" to "CREATE TABLE <table name>"
c.execute('''DROP TABLE fursealdata''') 
c.execute('''CREATE TABLE fursealdata
             (behaviour integer, filename text, range text, depth blob, x blob, 
             y blob, z blob, type blob, place blob)''')
conn.commit()

# iterate through files and insert information into the db file
dir_name = input("Directory name: ")
contents = os.listdir(dir_name) # the filenames

for i in contents:
    i = os.path.join(os.path.relpath(dir_name), i)
    if i.lower().endswith(('.csv')) == False: # nothing but CSVs should be getting by
        print('Non .csv found')
        continue
    
    data = pd.read_csv(i, sep=",", header=0, names=['depth', 'x', 'y', 'z', 'behaviour', 'type', 'place'], low_memory=False)
    # print(data)
    for behaviour in range(0, 27):
        temp = data.loc[data['behaviour'] == behaviour]
#         print(b, '=' * 20)
        if len(temp) > 0:
#             print(i)
            indexes = temp.index.tolist()
            groups = list(find_ranges(indexes))
            for r in groups:
#                 print(r) # prints ranges
                if type(r) is not type(groups[0]):
                    continue # protects against "ranges" that are not truly ranges
                x = data.iloc[r[0]:r[1]]['x']
                y = data.iloc[r[0]:r[1]]['y']
                z = data.iloc[r[0]:r[1]]['z']
                depth = data.iloc[r[0]:r[1]]['depth']
                t = data.iloc[r[0]:r[1]]['type']
                place = data.iloc[r[0]:r[1]]['place']
                c.execute("INSERT INTO fursealdata VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                          (behaviour, i, str(r), str(list(depth)), str(list(x)), 
                           str(list(y)), str(list(z)), str(list(t)), str(list(place))))
            conn.commit()
conn.close()

Directory name: csv


In [15]:
# CHANGE THE FOLLOWING LINE TO preferredDatabase.db
conn = sqlite3.connect('fursealdata.db') 
c = conn.cursor()

files_and_ranges = []


# CHANGE BEHAVIOR HERE (refer to dictionary in README)
for row in c.execute("SELECT * FROM fursealdata WHERE behaviour = 22"): 
    files_and_ranges.append(row)
    
    
conn.close()

In [16]:
# CHANGE THE FOLLOWING LINE TO preferredDatabase.db
conn = sqlite3.connect('fursealdata.db')
c = conn.cursor()

num_ranges = len(files_and_ranges) # number of ranges in the specified directory
temp = math.ceil(math.sqrt(num_ranges))
k = 0
for file in files_and_ranges: # 'file' is a misnomer; it's a single range out of all of them
    filename = file[1] # one filename per index (can be repeated amongst files_and_ranges)
    filename_for_fig = filename.replace(".csv", "")
    
    
    # CHANGE DIRECTORY TO PREVIOUSLY SPECIFIED DIRECTORY
    filename_for_fig = filename_for_fig.replace("csv/", "")
    
    b = file[0] # one behavior per index (will be repeated amongst files_and_ranges)
    range_for_axis = file[2] # 

    # COMMENT AND UNCOMMENT AS DESIRED
    depth_range = file[3]
    x_range = file[4]
    y_range = file[5]
    z_range = file[6]
#         t_range = file[7] # UNCOMMENT FOR TYPE TO BE GRAPHED; uncomment line 35 as well
#         place_range = file[8] # UNCOMMENT FOR PLACE TO BE GRAPHED; uncomment line 36 as well


    # grab data from database
    for char in '[,]()':
        range_for_axis = range_for_axis.replace(char,"")
        x_range = x_range.replace(char,"")
        y_range = y_range.replace(char, "")
        z_range = z_range.replace(char, "")
        depth_range = depth_range.replace(char, "")
#         t_range = t_range.replace(char, "")
#         place_range = place_range.replace(char, "")
        
        
    range_for_axis = list(map(int, range_for_axis.split()))
    x_range = list(map(float, x_range.split()))
    y_range = list(map(float, y_range.split()))
    z_range = list(map(float, z_range.split()))
    depth_range = list(map(float, depth_range.split()))
    
    # NOTE: png ratio can be changed (width, height) in inches
    plt.figure(dpi=300, figsize=[8, 5])
    fig_name = str(b) + " " + filename_for_fig + str(range_for_axis)
    
    plt.plot(range(range_for_axis[0], range_for_axis[1]), x_range, label='x', color='red')
    plt.plot(range(range_for_axis[0], range_for_axis[1]), y_range, label='y', color='blue')
    plt.plot(range(range_for_axis[0], range_for_axis[1]), z_range, label='z', color='green')
    plt.plot(range(range_for_axis[0], range_for_axis[1]), depth_range, label='depth', color='magenta')
    plt.title(fig_name)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    fig_name += '.png'
    
    # CHANGE TRANSPARENCY
    plt.savefig(fname=fig_name, transparent=True, format='png')
    
    plt.close()
            
        
conn.close()